In [1]:
import mlflow
import time
import torch

from omegaconf import OmegaConf
from torch.utils.data import DataLoader

from nba_betting_ai.consts import proj_paths
from nba_betting_ai.data.processing import add_prefix
from nba_betting_ai.model.bayesian import BayesianResultPredictor
from nba_betting_ai.model.inputs import prepare_scalers
from nba_betting_ai.training.dataset import NBADataset
from nba_betting_ai.training.pipeline import prepare_data
from nba_betting_ai.training.run import run_training

In [2]:
process_key = lambda key: "".join(
    word[0]
    for word in key.split('_')
)
def name_run(config):
    run_name = '_'.join(
        f"{value}{process_key(key)}"
        for key, value in config.training.model_config.items()
        if key != 'time_scaling'
    )
    run_name += f"_wd{config.training.params.weight_decay}"
    run_name += '_WL'  if config.training.params.weighted_loss else "_nWL"
    run_name += f"_TS" if config.training.model_config.time_scaling else "_nTS"
    run_name += '_' + config.model_name.split('_')[0]
    return run_name

In [ ]:
now = time.strftime("%Y%m%d-%H%M%S")
config = OmegaConf.load(proj_paths.config.default)
scalers_path = proj_paths.output / 'scalers.pkl'
input_teams = config['inputs_team'] or []
data_prepared = prepare_data(**config['data_params'])
scalers = prepare_scalers(data_prepared.X_train, input_teams, scalers_path)

train_dataset = NBADataset(input_teams, config['target'], data_prepared.X_train, data_prepared.teams, scalers)
test_dataset = NBADataset(input_teams, config['target'], data_prepared.X_test, data_prepared.teams, scalers)

params = config['training']['params']
train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_init = OmegaConf.create(dict(
    team_count=len(data_prepared.teams),
    team_features=len(input_teams)//2,
    **config['training']['model_config']
))
model_init_path = proj_paths.output / "model_init.yaml"
model_init_path.open("w").write(OmegaConf.to_yaml(model_init))
model = BayesianResultPredictor(**model_init).to(device)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=params['learning_rate'],
    weight_decay=params['weight_decay'] or 0.0
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = params.get('lr_gamma', 1.0))

create_experiment = mlflow.get_experiment_by_name(config['experiment_name']) is None
if create_experiment:
    mlflow.create_experiment(
        name=config['experiment_name'],
        tags={
            'author': 'Tom Bukic',
            'company': 'accelerate.ai'
        }
    )
    create_experiment = False

mlflow.set_experiment(config['experiment_name'])
with mlflow.start_run(run_name=f'{name_run(config)}-{now}'):
    mlflow.log_artifact(model_init_path)
    run_training(model, train_loader, test_loader, config, optimizer,  device, scheduler,  timestamp=now)

2025/02/11 15:03:50 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
loss: 1.5612 | epoch:  0/80:   0%|          | 75/20320 [00:04<09:30, 35.49it/s]  